<a href="https://colab.research.google.com/github/Rizzuto82/Esercizi-develhope/blob/main/esercizio__EU_superstore.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 4.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425344 sha256=53984b234b36f51cfca33a5ad6b70634e91c82543fc0b61a421c4baadb7decc8
  Stored in directory: /root/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built pyspark


In [2]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import when
import pyspark.sql.functions as func

In [3]:
spark = SparkSession.builder.master("local[*]").appName("Datamanipulation").getOrCreate()

In [4]:
spark

In [5]:
# read our data - lives in a csv file

df = spark.read.option("header","true").csv("/content/Sample - EU Superstore.csv",inferSchema = True)

In [6]:
df.printSchema()

root
 |-- Row ID: integer (nullable = true)
 |-- Order ID: string (nullable = true)
 |-- Order Date: string (nullable = true)
 |-- Ship Date: string (nullable = true)
 |-- Ship Mode: string (nullable = true)
 |-- Customer ID: string (nullable = true)
 |-- Customer Name: string (nullable = true)
 |-- Segment: string (nullable = true)
 |-- City: string (nullable = true)
 |-- State: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- Region: string (nullable = true)
 |-- Product ID: string (nullable = true)
 |-- Category: string (nullable = true)
 |-- Sub-Category: string (nullable = true)
 |-- Product Name: string (nullable = true)
 |-- Sales: double (nullable = true)
 |-- Quantity: integer (nullable = true)
 |-- Discount: double (nullable = true)
 |-- Profit: double (nullable = true)



In [7]:
df.show()

+------+---------------+----------+----------+--------------+-----------+----------------+-----------+-------------+--------------------+--------------+-------+---------------+---------------+------------+--------------------+--------+--------+--------+-------+
|Row ID|       Order ID|Order Date| Ship Date|     Ship Mode|Customer ID|   Customer Name|    Segment|         City|               State|       Country| Region|     Product ID|       Category|Sub-Category|        Product Name|   Sales|Quantity|Discount| Profit|
+------+---------------+----------+----------+--------------+-----------+----------------+-----------+-------------+--------------------+--------------+-------+---------------+---------------+------------+--------------------+--------+--------+--------+-------+
|     1|ES-2017-1311038|07/02/2017|11/02/2017|Standard Class|   AS-10045|  Aaron Smayling|  Corporate|        Leeds|             England|United Kingdom|  North|OFF-ST-10000988|Office Supplies|     Storage|Fellowes 

In [8]:
# how many rows of the EU Superstore dataset have the country being France
df.filter(df["Country"] == "France").count()

2827

In [9]:
# of those, how many are profitable?
df.filter((df["Country"] == "France") & (df["Profit"] > 0)).count()


2329

In [10]:
# how many different discount backets exist? what are they?
discount_backets = df.select("Discount").distinct().count()
discount_backets

14

In [14]:
# let's see the total profit by discount bracket, make sure they are ordered by   (raggruppa per sconto e usa sconto su profitto)
df2 = df.groupBy('Discount').agg({'Profit' : 'sum'}).orderBy('Discount', ascending=False)
df2.show()

+--------+-------------------+
|Discount|        sum(Profit)|
+--------+-------------------+
|    0.85|          -3068.658|
|     0.8|           -460.284|
|     0.7|          -5496.765|
|    0.65| -6221.965499999999|
|     0.6|-20517.456000000002|
|     0.5|         -96632.115|
|    0.45|         -1103.1915|
|     0.4|-21346.427999999996|
|    0.35|          -9122.649|
|     0.3| -758.4209999999999|
|     0.2| 2189.5499999999984|
|    0.15| 24677.563499999975|
|     0.1|  126884.0309999999|
|     0.0| 383806.53000000026|
+--------+-------------------+



In [ ]:
# what is the value after which we should stop offering discount?  qual è il valore più grande d sconto che porta profitto positivo



In [16]:
# who are the top 5 most profitable customers
top_5_customer = df.groupBy("Customer Name").agg({"Profit":"sum"}).orderBy("sum(Profit)",ascending=False).limit(5)
top_5_customer.show()

+-----------------+------------------+
|    Customer Name|       sum(Profit)|
+-----------------+------------------+
|     Susan Pistek| 4974.512999999999|
|    Patrick Jones|3986.0039999999995|
|Patrick O'Donnell|          3778.197|
|    Ellis Ballard|           3459.66|
|  Mike Gockenbach|3144.4439999999995|
+-----------------+------------------+



In [19]:
# get all the rows belonging to those 5 customer names: hint, you may need the collect method - how many rows are they?
 #seleziona colonna costumer applica collect, indicizza primi 5,filtra la colonna d customer  isin ...costumer count




In [20]:
# create a new column which is the value of the sale were there not discount applied. Hint: new colonna = sales/(1-d)
no_discount = df.filter(df["Discount"]==0).count()
df = df.withColumn("Original", df["Sales"]/no_discount)
df.show()

+------+---------------+----------+----------+--------------+-----------+----------------+-----------+-------------+--------------------+--------------+-------+---------------+---------------+------------+--------------------+--------+--------+--------+-------+--------------------+
|Row ID|       Order ID|Order Date| Ship Date|     Ship Mode|Customer ID|   Customer Name|    Segment|         City|               State|       Country| Region|     Product ID|       Category|Sub-Category|        Product Name|   Sales|Quantity|Discount| Profit|            Original|
+------+---------------+----------+----------+--------------+-----------+----------------+-----------+-------------+--------------------+--------------+-------+---------------+---------------+------------+--------------------+--------+--------+--------+-------+--------------------+
|     1|ES-2017-1311038|07/02/2017|11/02/2017|Standard Class|   AS-10045|  Aaron Smayling|  Corporate|        Leeds|             England|United Kingdom

In [21]:
# calculate the difference between sales and discount value
df=df.withColumn("difference", df["Sales"] - df["Discount"])
df.show()

+------+---------------+----------+----------+--------------+-----------+----------------+-----------+-------------+--------------------+--------------+-------+---------------+---------------+------------+--------------------+--------+--------+--------+-------+--------------------+------------------+
|Row ID|       Order ID|Order Date| Ship Date|     Ship Mode|Customer ID|   Customer Name|    Segment|         City|               State|       Country| Region|     Product ID|       Category|Sub-Category|        Product Name|   Sales|Quantity|Discount| Profit|            Original|        difference|
+------+---------------+----------+----------+--------------+-----------+----------------+-----------+-------------+--------------------+--------------+-------+---------------+---------------+------------+--------------------+--------+--------+--------+-------+--------------------+------------------+
|     1|ES-2017-1311038|07/02/2017|11/02/2017|Standard Class|   AS-10045|  Aaron Smayling|  Co

In [ ]:
# how much money did we not gain due to the discounts - per discount bracket? raggruppando x discount quanto abbiamo perso?


In [ ]:
# find the discount bracket which made us not gain the most (dynamically)



In [ ]:
# what would have been the total profit if we removed all orders from that discount group?


In [ ]:
#how much more (or less) profit is that?



In [22]:
# create a temporary table for our superstore table in sql
from pyspark.sql.types import StringType, StructField, IntegerType, StructType


In [23]:
df = spark.read.csv("/content/Sample - EU Superstore.csv", header=True, inferSchema=True)


In [24]:
df.createOrReplaceTempView("TABELLA")

In [25]:
df.show()

+------+---------------+----------+----------+--------------+-----------+----------------+-----------+-------------+--------------------+--------------+-------+---------------+---------------+------------+--------------------+--------+--------+--------+-------+
|Row ID|       Order ID|Order Date| Ship Date|     Ship Mode|Customer ID|   Customer Name|    Segment|         City|               State|       Country| Region|     Product ID|       Category|Sub-Category|        Product Name|   Sales|Quantity|Discount| Profit|
+------+---------------+----------+----------+--------------+-----------+----------------+-----------+-------------+--------------------+--------------+-------+---------------+---------------+------------+--------------------+--------+--------+--------+-------+
|     1|ES-2017-1311038|07/02/2017|11/02/2017|Standard Class|   AS-10045|  Aaron Smayling|  Corporate|        Leeds|             England|United Kingdom|  North|OFF-ST-10000988|Office Supplies|     Storage|Fellowes 

In [26]:
# use an SQL query to count the number of rows
spark.sql("SELECT * FROM TABELLA").count()


10000

In [ ]:
# Use an SQL query to calculate the profit ratio for each country: hint, ratio is sum(profit)/sum(sales)
spark.sql( SELECT Country

In [ ]:
# is the country with the largest profit ratio, the country with the largest profit?

